In [ ]:
import pymc3 as pm
import numpy as np
import theano
import theano.tensor as tt
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
from datetime import timedelta

from tvb_inversion.pymc3.plot import plot_posterior_samples
from tvb_inversion.base.diagnostics import (zscore, shrinkage)
from tvb_inversion.pymc3.plot import (
    plot_posterior_samples_model_parameters,
    plot_posterior_samples_global_parameters
)

%load_ext autoreload
%autoreload 2

In [ ]:
# run_id = "2023-01-12_1752"  # around ground truth
# run_id = "2023-01-13_1919"  # shifted from ground truth
# run_id = "2023-01-17_1710"  # shifted from ground truth
# run_id = "2023-01-18_1755"  # shifted from ground truth
# run_id = "2023-01-24_1712"  # shifted from ground truth
# run_id = "2023-01-25_1332"  # shifted from ground truth
run_id = "2023-02-10_1605"  # around ground truth

inference_data = az.from_netcdf(f"pymc3_data/{run_id}_idata.nc")
inference_summary = pd.read_json(f"pymc3_data/{run_id}_isummary.json")
with open(f"pymc3_data/{run_id}_iparams.json", "r") as f:
    inference_params = json.load(f)
    f.close()
with open(f"pymc3_data/{run_id}_sim_params.json", "r") as f:
    sim_params = json.load(f)
    f.close()

In [ ]:
inference_params

In [ ]:
X = inference_data.observed_data.x_obs.values

In [ ]:
chains = len(inference_data.sample_stats.chain)
draws = len(inference_data.sample_stats.draw)

In [ ]:
print("#chains: ", chains)
print("#draws: ", draws)
divergent = inference_data.sample_stats["diverging"].values
print("Number of Divergent: %d" % divergent.nonzero()[0].size)
divperc = (divergent.nonzero()[0].size / (chains * draws)) * 100
print("Percentage of Divergent: %.1f" % divperc)
print("Mean tree accept: %.2f" % inference_data.sample_stats.acceptance_rate.values.mean())
print("Mean tree depth: %.2f" % inference_data.sample_stats.tree_depth.values.mean())
print("Sampling time:", str(timedelta(seconds=inference_data.sample_stats.sampling_time)))

In [ ]:
inference_summary.loc[
    ["model_a_star", "nsig_star",
     "amplitude_star", "offset_star", "measurement_noise_star"]]

In [ ]:
ground_truth = np.array(
    [sim_params["model_a"], sim_params["nsig"], 1.0, 0.0, 0.0])

inference_summary_ = inference_summary.loc[
    ["model_a", "nsig",
     "amplitude", "offset", "measurement_noise"]]

inference_summary_.insert(0, "ground_truth", ground_truth)
inference_summary_

In [ ]:
df = pd.DataFrame(
    data=inference_data.posterior.model_a.values.flatten(),
    columns=["Region 1"]
)

plot_posterior_samples_model_parameters(df, sim_params["model_a"])

In [ ]:
data={
    "nsig": inference_data.posterior.nsig.values.flatten(),
    "measurement_noise": inference_data.posterior.measurement_noise.values.flatten(),
}
df = pd.DataFrame(data)

plot_posterior_samples_global_parameters(df, sim_params)

In [ ]:
posterior_x_obs = inference_data.posterior_predictive.x_obs.values.reshape(
    (chains*draws, *X.shape[:-1]))

In [ ]:
f3 = plt.figure(figsize=(10,5))
plt.plot(np.percentile(posterior_x_obs[:, :, 0], [2.5, 97.5], axis=0).T,
              "k", label=r"$V_{95\% PP}(t)$")
plt.plot(X[:, 0, 0], label="V_observed")
plt.legend(fontsize=16)
plt.xlabel("time (ms)", fontsize=16)
plt.tick_params(axis="both", labelsize=16)

plt.show()

In [ ]:
posterior_dWt_star = inference_data.posterior.dWt_star.values[:, :, :, 0, :].reshape(
    (chains*draws, *X.shape))

In [ ]:
inference_summary.loc[
    inference_summary.index.str.contains("dWt_star")].mean()

In [ ]:
f10 = plt.figure(figsize=(10,5))
plt.plot(np.percentile(posterior_dWt_star[:, :, 0, 0], [2.5, 97.5], axis=0).T,
         "k", label=r"$V_{95\% PP}(t)$")
plt.legend(fontsize=16)
plt.xlabel("time (ms)", fontsize=16)
plt.tick_params(axis="both", labelsize=16)

plt.show()

In [ ]:
waic = az.waic(inference_data)
loo = az.loo(inference_data)
print("WAIC: ", waic.waic)
print("LOO: ", loo.loo)

In [ ]:
def get_posterior_mean(idata, params):
    posterior = np.asarray([idata.posterior[param].values.reshape((idata.posterior[param].values.size,)) for param in params])
    return posterior.mean(axis=1)

def get_posterior_std(idata, params):
    posterior = np.asarray([idata.posterior[param].values.reshape((idata.posterior[param].values.size,)) for param in params])
    return posterior.std(axis=1)

In [ ]:
# zscores
posterior_mean = get_posterior_mean(inference_data, ["model_a", "nsig"])

posterior_std = get_posterior_std(inference_data, ["model_a", "nsig"])

ground_truth = np.array([sim_params["model_a"], sim_params["nsig"]])

zscores = zscore(ground_truth, posterior_mean, posterior_std)

In [ ]:
# shrinkages
posterior_std = get_posterior_std(inference_data, ["model_a_star", "nsig_star"])

prior_std = np.ones((2,))

shrinkages = shrinkage(prior_std, posterior_std)

In [ ]:
def zscore_gt(true_mean, post_mean):
    return np.abs((post_mean - true_mean) / true_mean)

def zscore_prior(true_mean, post_mean, prior_std):
    return np.abs((post_mean - true_mean) / prior_std)

In [ ]:
zscores_gt = zscore_gt(ground_truth, posterior_mean)
zscores_prior = zscore_prior(ground_truth, posterior_mean, 0.5 * ground_truth)

In [ ]:
f4 = plt.figure(figsize=(12,8))
plt.plot(shrinkages[0], zscores[0], color="blue", linewidth=0, marker="*", markersize=12, label="model_a")
plt.plot(shrinkages[1], zscores[1], color="green", linewidth=0, marker="*", markersize=12, label="nsig")
plt.xlabel("posterior shrinkage")
plt.ylabel("posterior zscore")
plt.legend(fontsize=16)
plt.plot();

In [ ]:
f5 = plt.figure(figsize=(12,8))
plt.plot(shrinkages[0], zscores_gt[0], color="blue", linewidth=0, marker="*", markersize=12, label="model_a")
plt.plot(shrinkages[1], zscores_gt[1], color="green", linewidth=0, marker="*", markersize=12, label="nsig")
plt.xlabel("posterior shrinkage")
plt.ylabel("posterior zscore")
plt.legend(fontsize=16)
plt.plot();

In [ ]:
f6 = plt.figure(figsize=(12,8))
plt.plot(shrinkages[0], zscores_prior[0], color="blue", linewidth=0, marker="*", markersize=12, label="model_a")
plt.plot(shrinkages[1], zscores_prior[1], color="green", linewidth=0, marker="*", markersize=12, label="nsig")
plt.xlabel("posterior shrinkage")
plt.ylabel("posterior zscore")
plt.legend(fontsize=16)
plt.plot();